

И так , у нас есть данные о 90 189 игроках, которые установили игру во время проведения AB-теста. Переменные включают:

userid: Уникальный номер, идентифицирующий каждого игрока.

version: Определяет, был ли игрок в контрольной группе (gate_30 - препятствие на уровне 30) или в группе с перемещенным препятствием (gate_40 - препятствие на уровне 40).

sum_gamerounds: Количество раундов, сыгранных игроком в течение первых 14 дней после установки.

retention_1: Возвратился ли игрок и сыграл 1 день после установки?

retention_7: Возвратился ли игрок и сыграл 7 дней после установки?

Когда игрок устанавливал игру, он или она были случайно назначены в одну из групп.

Шаг 1: Определение проблемы
Мы стремимся понять разницу в поведении пользователей между двумя версиями (gate_30 и gate_40) игры Cookie Cats. В частности, мы проанализируем переменную "sum_gamerounds", представляющую количество сыгранных раундов пользователями.

Шаг 2: Формулировка гипотез
Нулевая гипотеза (H0): Нет статистически значимой разницы в количестве сыгранных раундов между двумя версиями.

Альтернативная гипотеза (H1): Существует статистически значимая разница в количестве сыгранных раундов между двумя версиями.

Шаг 3: Исследование данных
Давайте начнем с загрузки набора данных и изучения основной статистической информации:

Загрузка набора данных:
В этом разделе мы загружаем набор данных Cookie Cats в pandas dataframe.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import shapiro, levene, ttest_ind, mannwhitneyu
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/kaggle/input/mobile-games-ab-testing-cookie-cats/cookie_cats.csv' )
df.info()

In [ ]:
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head())
    print("##################### Tail #####################")
    print(dataframe.tail())
    print("##################### NA #####################")
    print(dataframe.isnull().sum())

check_df(df)

In [ ]:
df.describe().T

In [ ]:
# Number of unique users in groups A and B.

df.groupby("version")[["userid"]].nunique()

In [ ]:
# Some statistics of "sum_gamerounds" variable by "versions"
df.groupby("version").agg({"sum_gamerounds" : ["mean","max","count"]})

In [ ]:
# Visualize the differences between groups
sns.countplot(x='version', data=df)
plt.title('Player Count by Version')
plt.show()

Проверка предположения о нормальности (тест Шапиро-Уилка)
Давайте проверим предположение о нормальности для переменной "sum_gamerounds" в обеих версиях:

H0: Предположение о нормальном распределении выполняется.

H1: Предположение о нормальном распределении не выполняется.

p-значение < 0,05 => H0 -> Отклоняем

p-значение > 0,05 => H0 -> Не можем отклонить

In [ ]:
# Normality check for gate_30
test_stat, pvalue = shapiro(df.loc[df["version"] == "gate_30", "sum_gamerounds"])
print("Test Stat = %.4f, p-value = %.4f" % (test_stat, pvalue))

# Normality check for gate_40
test_stat, pvalue = shapiro(df.loc[df["version"] == "gate_40", "sum_gamerounds"])
print("Test Stat = %.4f, p-value = %.4f" % (test_stat, pvalue))

В обоих случаях p-значение = 0. Предположение о нормальности было отвергнуто, поскольку оно менее 0,05.

Проверка однородности дисперсии (тест Левена)
Давайте проверим однородность дисперсии переменной "sum_gamerounds" между двумя версиями:

H0: Дисперсия является однородной.

H1: Дисперсия не является однородной.

p-значение < 0.05 => H0 -> Отклонить

p-значение > 0.05 => H0 -> Не отклонять

In [ ]:
test_stat, pvalue = levene(df.loc[df["version"] == "gate_30", "sum_gamerounds"],
                            df.loc[df["version"] == "gate_40", "sum_gamerounds"])
print("Test Stat = %.4f, p-value = %.4f" % (test_stat, pvalue))

значение p-уровня значимости больше 0,05. Таким образом, нулевая гипотеза не может быть отвергнута.

In [ ]:
Выбор между параметрическим или непараметрическим тестом

Если выполняются предположения о нормальности и однородности, можно применить параметрический тест, например, T-тест. В противном случае, более подходящим будет непараметрический тест, такой как тест Манна-Уитни U.

Учитывая, что предположение о нормальности нарушается как для версии gate_30, так и gate_40, а предположение об однородности дисперсий выполняется, рекомендуется выбрать непараметрический тест. В частности, тест Манна-Уитни U подходит для сравнения распределения переменной "sum_gamerounds" между двумя версиями.

Почему выбрать тест Манна-Уитни U:
1. Нарушение предположения о нормальности:

Поскольку предположение о нормальности не выполняется, тест Манна-Уитни U не зависит от предположения о нормальном распределении и считается устойчивой альтернативой.

2. Однородность дисперсий:

Тест Манна-Уитни U не требует равенства дисперсий, поэтому он подходит даже при различии дисперсий между группами.

Применение теста Манна-Уитни U:
Давайте проведем тест Манна-Уитни U для переменной "sum_gamerounds" между версиями gate_30 и gate_40.

Интерпретация результатов теста Манна-Уитни:
Если p < 0.05: Отвергнуть нулевую гипотезу (H0); существует статистически значимая разница в распределении "sum_gamerounds" между gate_30 и gate_40.

Если p > 0.05: Не отвергнуть нулевую гипотезу (H0); нет статистически значимой разницы в распределении "sum_gamerounds" между gate_30 и gate_40.

Учитывая нарушение предположений о нормальности, тест Манна-Уитни предоставляет надежный подход для сравнения центральных тенденций двух групп без полагания на предположения о нормальном распределении.


In [ ]:
test_stat, pvalue = mannwhitneyu(df.loc[df["version"] == "gate_30", "sum_gamerounds"],
                            df.loc[df["version"] == "gate_40", "sum_gamerounds"])
print("Test Stat = %.4f, p-value = %.4f" % (test_stat, pvalue))

Согласно результатам теста Манна-Уитни, статистика теста составляет 1024331250.5000, а p-значение равно 0.0502. В данном случае, так как p-значение меньше 0.05, нулевая гипотеза (H0) отвергается, указывая на статистически значимую разницу в распределении переменной "sum_gamerounds" между версиями "gate_30" и "gate_40".

Вывод:
Тест Манна-Уитни, проведенный для сравнения количества сыгранных раундов между версиями игры Cookie Cats "gate_30" и "gate_40", показывает статистически значимую разницу между группами (p = 0.0502). Этот результат говорит о возможных влияниях версий на поведение пользователей.
если мы хотим поддерживать высокий уровень удержания — как 1-дневный, так и 7-дневный — нам не следует перемещать порог с уровня 30 на уровень 40. Конечно, есть и другие показатели, на которые мы могли бы обратить внимание, например количество сыгранных игровых раундов или количество внутриигровых покупок, совершенных двумя AB-группами. Но удержание — один из наиболее важных показателей.